In [1]:
import os
import mlflow
import numpy as np
import pandas as pd

from math import sqrt
from joblib import delayed
from joblib import Parallel

from warnings import catch_warnings
from warnings import filterwarnings
from multiprocessing import cpu_count
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce
from datetime import datetime

from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


# Data

In [2]:
df = pd.read_csv('./../final_dfs/for_modeling/df_final_date_wide_2007.csv', converters={'ags2': str, 'ags5': str} )
df.set_index('ags5', drop=True, inplace=True)
df = df.drop("Unnamed: 0", axis=1)
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df[(df['date'] >= '2010-01-01') & (df['date'] < '2020-01-01')]
df = df.rename(columns={'date': 'ds', 'unemployment_rate': 'y'})

# 401 Model for each kreis

In [3]:
# one-step forecast
def last_obs(history):
    res = history.iloc[-24:]["y"]
    return res

In [4]:
config = ['linear', 15.0, True, 'additive']
last_obs(df)

ags5
16076    5.3
16076    5.2
16076    4.9
16076    4.6
16076    4.5
16076    4.4
16076    4.5
16076    4.5
16076    4.4
16076    4.4
16076    4.4
16076    4.6
16077    8.0
16077    8.0
16077    7.6
16077    7.2
16077    7.1
16077    7.0
16077    7.0
16077    7.0
16077    6.5
16077    6.5
16077    6.3
16077    6.5
Name: y, dtype: float64

In [5]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [6]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [7]:
(train_test_split(df, 24)[1] )

,ags2,ds,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,number_of_companies_construction,...,realized_short_time_work_companies,realized_short_time_work_people,underemployment_without_short_time _work,unemployment_benefit_entitled,unemployment_benefit_recipients,y,registerd_jobs,unemployed,employees_social_security_at_work,employees_social_security_at_residence
ags5,,,,,,,,,,,,,,,,,,,,,
16076,16,2019-01-01,4.0,6.0,9.0,99.0,158.0,131.0,48.0,773.0,...,NaN,NaN,3454.0,4494.0,1029.0,5.3,898,2633,29989,37861
16076,16,2019-02-01,6.0,4.0,7.0,-99.0,-99.0,-99.0,-99.0,-99.0,...,NaN,NaN,3435.0,4497.0,1042.0,5.2,901,2605,29965,37904
16076,16,2019-03-01,2.0,1.0,10.0,105.0,141.0,103.0,51.0,685.0,...,NaN,NaN,3313.0,4465.0,918.0,4.9,959,2451,30043,37979
16076,16,2019-04-01,2.0,3.0,5.0,121.0,117.0,135.0,44.0,682.0,...,NaN,NaN,3209.0,4434.0,822.0,4.6,912,2307,30129,38029
16076,16,2019-05-01,3.0,1.0,2.0,125.0,154.0,158.0,58.0,803.0,...,NaN,NaN,3091.0,4394.0,781.0,4.5,923,2216,30146,38023
16076,16,2019-06-01,1.0,0.0,2.0,125.0,154.0,158.0,59.0,803.0,...,NaN,NaN,3039.0,4327.0,775.0,4.4,987,2151,30145,38028
16076,16,2019-07-01,3.0,1.0,3.0,125.0,155.0,158.0,59.0,805.0,...,NaN,NaN,3098.0,4272.0,838.0,4.5,930,2192,30117,37888
16076,16,2019-08-01,4.0,1.0,3.0,125.0,155.0,158.0,59.0,806.0,...,NaN,NaN,3072.0,4158.0,849.0,4.5,829,2209,30208,38103
16076,16,2019-09-01,2.0,1.0,4.0,122.0,149.0,137.0,56.0,770.0,...,NaN,NaN,3043.0,4115.0,812.0,4.4,751,2149,30306,38262


In [10]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model and make forecast for history
    yhat = last_obs(train)
    # estimate prediction error
    error = measure_rmse(test['y'], yhat)
    return error

In [11]:
def run_model(data, n_test): 
    with mlflow.start_run(run_name="baseline"):
        ags5 = list(set(df.index.values.tolist()))
        rmse = []
        for i in ags5:
            df_ags5 = df.filter(regex=i, axis=0)
            current_rmse = walk_forward_validation(df_ags5, n_test)
            print(current_rmse)
            rmse.append(current_rmse)

        # Log params
        params = {"growth":"NONE_two"}

        mlflow.log_params(params)

        # Log metrics
        mlflow.log_metric('rmse', np.mean(rmse))
        res = pd.DataFrame(
                            {'ags5': ags5,
                             'rmse': rmse,
                            })
        return res

In [12]:
res = run_model(df, 24)

0.4743416490252569
1.5098289086294958
0.383514449966795
1.1652968148358882
0.5392896562454479
0.5318991132410982
0.8621678104251708
0.13994046353122216
1.2489995996796799
0.6096447052723962
0.3999999999999999
0.21015867021530815
0.8215838362577492
0.582380173655205
0.6528782428600298
1.1601005703529903
1.1530755974060563
0.9565563234854496
0.733143914930759
0.6525590650150632
0.5496210815947049
0.4462809279665295
0.627162924074226
0.9420721840708385
1.7492855684535902
0.44814432199162507
0.67700320038633
0.36742346141747667
0.3109126351029605
2.0238165265985293
2.2955573034305488
0.7802243267163618
1.3250786140200639
1.3343225497107762
0.7035623639735145
0.893261813057441
0.5488624600025038
0.7155999813676167
1.2869537676233753
1.2026013470805694
0.44860896112315896
0.5940678973540539
0.8753570700005797
1.006023525238517
0.5996526772502008
0.6327848502189877
0.8913940393189386
1.123424823178362
0.5556527692723217
0.8213302218555781
1.0338036564067665
0.9356370378873781
1.15307559740605

In [11]:
res

,ags5,rmse
0,12070,1.601822
1,7235,0.447679
2,9773,0.317543
3,9673,0.320156
4,8212,0.565685
...,...,...
396,3460,0.561249
397,9564,0.660177
398,9777,0.267706
399,9464,1.404902


In [23]:
#res.to_csv("naive_forcasting_rmse.csv")
